In [ ]:
import pandas as pd
import os

### LOAD DATA
umsatz_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/umsatzdaten_gekuerzt.csv"
wetter_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/wetter.csv"
kiwo_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/kiwo.csv"

# Load external data
umsatz_df = pd.read_csv(umsatz_url)
wetter_df = pd.read_csv(wetter_url)
kiwo_df = pd.read_csv(kiwo_url)

umsatz_df['Datum'] = pd.to_datetime(umsatz_df['Datum'])
wetter_df['Datum'] = pd.to_datetime(wetter_df['Datum'])
kiwo_df['Datum'] = pd.to_datetime(kiwo_df['Datum'])

print("Umsatz columns:", umsatz_df.columns.tolist())
print("Wetter columns:", wetter_df.columns.tolist())
print("KiWo columns:", kiwo_df.columns.tolist())


### MERGE DATA
merged_df = pd.merge(umsatz_df, wetter_df, on="Datum", how="left")
merged_df = pd.merge(merged_df, kiwo_df, on="Datum", how="left")
merged_df = merged_df.sort_values('Datum')

print("\nFinal merged dataframe shape:", merged_df.shape)
print("\nFinal columns:", merged_df.columns.tolist())
print("\nFirst few rows:")
print(merged_df.head())

merged_df.to_csv("merged_output.csv", index=False)


### MISSING VALUE HANDLING''
merged_df = merged_df.set_index("Datum")
merged_df["Temperatur"] = merged_df["Temperatur"].interpolate(method="time")
merged_df["Windgeschwindigkeit"] = merged_df["Windgeschwindigkeit"].interpolate(method="time")
merged_df["Bewoelkung"] = merged_df["Bewoelkung"].interpolate(method="time")
merged_df = merged_df.reset_index()

# Fill missing values for Kieler
merged_df["KielerWoche"] = merged_df["KielerWoche"].fillna(0)

# Fill missing values for Wettercode
merged_df["Wettercode"] = (merged_df["Wettercode"].fillna(method="ffill").fillna(method="bfill"))

# Show the final merged result
print("\nFinal merged dataframe shape:", merged_df.shape)
print("\nFinal columns:", merged_df.columns.tolist())
print("\nFirst few rows:")
print(merged_df.head())

### ADD NEW VARIABLES
import holidays
de_holidays = holidays.Germany(years=range(2012, 2021))
hol_df = pd.DataFrame(list(de_holidays.items()), columns=["Datum", "Feiertag"]).assign(Datum=lambda df: pd.to_datetime(df["Datum"]),is_holiday=1)
merged_df = pd.merge(merged_df, hol_df[['Datum', 'Feiertag']], on="Datum", how="left")
merged_df["Feiertag"] = merged_df["Feiertag"].fillna("Kein Feiertag")


import yfinance as yf1
dax_df = yf1.download("^GDAXI", start="2012-01-01", end="2020-12-31")["Close"]
dax_df = (dax_df.reset_index().rename(columns={'Date': 'Datum','Close': 'DAX_Close'}))
# rename column ^GDAXI to DAX_Close
dax_df = dax_df.rename(columns={"^GDAXI": "DAX"})
merged_df = merged_df.merge(dax_df, on="Datum", how="left")
merged_df["DAX"] = (merged_df["DAX"].fillna(method="ffill").fillna(method="bfill"))

/tmp/ipykernel_33415/1051930130.py:47: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df["Wettercode"] = (merged_df["Wettercode"].fillna(method="ffill").fillna(method="bfill"))
/tmp/ipykernel_33415/1051930130.py:64: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dax_df = yf1.download("^GDAXI", start="2012-01-01", end="2020-12-31")["Close"]
[*********************100%***********************]  1 of 1 completed

Umsatz columns: ['id', 'Datum', 'Warengruppe', 'Umsatz']
Wetter columns: ['Datum', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode']
KiWo columns: ['Datum', 'KielerWoche']

Final merged dataframe shape: (9334, 9)

Final columns: ['id', 'Datum', 'Warengruppe', 'Umsatz', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode', 'KielerWoche']

First few rows:
           id      Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0     1307011 2013-07-01            1  148.828353         6.0     17.8375   
3638  1307013 2013-07-01            3  201.198426         6.0     17.8375   
7223  1307015 2013-07-01            5  317.475875         6.0     17.8375   
1819  1307012 2013-07-01            2  535.856285         6.0     17.8375   
5457  1307014 2013-07-01            4   65.890169         6.0     17.8375   

      Windgeschwindigkeit  Wettercode  KielerWoche  
0                    15.0        20.0          NaN  
3638                 15.0        20.0          Na


/tmp/ipykernel_33415/1051930130.py:67: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df["^GDAXI"] = (merged_df["^GDAXI"].fillna(method="ffill").fillna(method="bfill"))


In [10]:
# dieser code ghier soll die basic infos von wetter_df printen
print(wetter_df.head())  # Print first few rows to verify
print(wetter_df.info())  # Print info about the DataFrame
print(wetter_df.describe())  # Print descriptive statistics

# find values in wetter_df which have missing values or NaN
missing_values = wetter_df.isnull().sum()
print("\nMissing values in wetter_df:")
print(missing_values[missing_values > 0])  # Only print columns with missing values

# for all missing values take the value from the previous day
wetter_df.fillna(method='ffill', inplace=True)
print(wetter_df.head())  # Print first few rows to verify

       Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode
0 2012-01-01         8.0      9.8250                   14        58.0
1 2012-01-02         7.0      7.4375                   12         NaN
2 2012-01-03         8.0      5.5375                   18        63.0
3 2012-01-04         4.0      5.6875                   19        80.0
4 2012-01-05         6.0      5.3000                   23        80.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2601 entries, 0 to 2600
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Datum                2601 non-null   datetime64[ns]
 1   Bewoelkung           2591 non-null   float64       
 2   Temperatur           2601 non-null   float64       
 3   Windgeschwindigkeit  2601 non-null   int64         
 4   Wettercode           1932 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 101.7 KB
None

/tmp/ipykernel_4492/2195552814.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  wetter_df.fillna(method='ffill', inplace=True)


In [28]:
import pandas as pd
from datetime import datetime

# 1. Einlesen
df = pd.read_csv("merged_output.csv")

# 2. Datum parsen und sortieren
df['Datum'] = pd.to_datetime(df['Datum'], errors='coerce')
df = df.sort_values('Datum')

# 3. Übersicht der Spalten
print("Spaltenübersicht:", df.columns.tolist())
print(df.head(3))

print("\nNaN-Übersicht (in %):")
print((df.isna().sum() / len(df) * 100).round(1).sort_values(ascending=False))

# 4. Nur Zeilen mit Umsatz behalten (Training only!)
df = df[df['Umsatz'].notna()].copy()

df['Wochentag'] = df['Datum'].dt.dayofweek  # 0 = Montag, 6 = Sonntag
df['Wochenende'] = df['Wochentag'].isin([5,6]).astype(int)

train_start = datetime(2013, 7, 1)
train_end = datetime(2017, 7, 31)
val_start = datetime(2017, 8, 1)
val_end = datetime(2018, 7, 31)
test_start = datetime(2018, 8, 1)
test_end = datetime(2019, 7, 31)

# Jetzt filtern
train_set = df[(df['Datum'] >= train_start) & (df['Datum'] <= train_end)].copy()
val_set   = df[(df['Datum'] >= val_start)   & (df['Datum'] <= val_end)].copy()
test_set  = df[(df['Datum'] >= test_start)  & (df['Datum'] <= test_end)].copy()

Spaltenübersicht: ['id', 'Datum', 'Warengruppe', 'Umsatz', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode', 'KielerWoche']
           id      Datum  Warengruppe  Umsatz  Bewoelkung  Temperatur  \
0     1808011 2018-08-01            1     NaN         0.0     23.7625   
710   1808013 2018-08-01            3     NaN         0.0     23.7625   
1065  1808014 2018-08-01            4     NaN         0.0     23.7625   

      Windgeschwindigkeit  Wettercode  KielerWoche  
0                    10.0         0.0          NaN  
710                  10.0         0.0          NaN  
1065                 10.0         0.0          NaN  

NaN-Übersicht (in %):
Umsatz                 100.0
KielerWoche             97.5
Wettercode              18.4
Windgeschwindigkeit      3.6
Bewoelkung               3.6
Temperatur               3.6
id                       0.0
Datum                    0.0
Warengruppe              0.0
dtype: float64


In [12]:
import pandas as pd
from datetime import datetime

# Load the merged dataset from the first script
merged_df = pd.read_csv("merged_output.csv")

# Convert Datum column to datetime if not already done
merged_df['Datum'] = pd.to_datetime(merged_df['Datum'])

# Define date ranges for splitting
train_start = datetime(2013, 7, 1)
train_end = datetime(2017, 7, 31)
val_start = datetime(2017, 8, 1)
val_end = datetime(2018, 7, 31)
test_start = datetime(2018, 8, 1)
test_end = datetime(2019, 7, 31)

# Split the dataset based on date ranges
train_set = merged_df[(merged_df['Datum'] >= train_start) & 
                      (merged_df['Datum'] <= train_end)].copy()

validation_set = merged_df[(merged_df['Datum'] >= val_start) & 
                          (merged_df['Datum'] <= val_end)].copy()

test_set = merged_df[(merged_df['Datum'] >= test_start) & 
                     (merged_df['Datum'] <= test_end)].copy()

# Display information about the splits
print("Dataset Split Summary:")
print("=" * 50)
print(f"Original dataset shape: {merged_df.shape}")
print(f"Date range: {merged_df['Datum'].min()} to {merged_df['Datum'].max()}")
print()

print(f"Training set shape: {train_set.shape}")
print(f"Training date range: {train_set['Datum'].min()} to {train_set['Datum'].max()}")
print(f"Training period: 01.07.2013 to 31.07.2017")
print()

print(f"Validation set shape: {validation_set.shape}")
print(f"Validation date range: {validation_set['Datum'].min()} to {validation_set['Datum'].max()}")
print(f"Validation period: 01.08.2017 to 31.07.2018")
print()

print(f"Test set shape: {test_set.shape}")
print(f"Test date range: {test_set['Datum'].min()} to {test_set['Datum'].max()}")
print(f"Test period: 01.08.2018 to 31.07.2019")
print()

# Check for any missing dates or gaps
total_expected_days = (test_end - train_start).days + 1
total_actual_days = len(train_set) + len(validation_set) + len(test_set)
print(f"Expected total days: {total_expected_days}")
print(f"Actual total days in splits: {total_actual_days}")

# Save the split datasets
train_set.to_csv("train_set.csv", index=False)
validation_set.to_csv("validation_set.csv", index=False)
test_set.to_csv("test_set.csv", index=False)

print("\nDatasets saved:")
print("- Training set: 'train_set.csv'")
print("- Validation set: 'validation_set.csv'")
print("- Test set: 'test_set.csv'")

# Display first few rows of each set for verification
print("\n" + "="*50)
print("TRAINING SET - First 5 rows:")
print(train_set.head())

print("\n" + "="*50)
print("VALIDATION SET - First 5 rows:")
print(validation_set.head())

print("\n" + "="*50)
print("TEST SET - First 5 rows:")
print(test_set.head())

# Optional: Display some statistics about each dataset
print("\n" + "="*50)
print("DATASET STATISTICS:")
print("\nTraining Set Statistics:")
print(train_set.describe())

print("\nValidation Set Statistics:")
print(validation_set.describe())

print("\nTest Set Statistics:")
print(test_set.describe())

Dataset Split Summary:
Original dataset shape: (4089, 9)
Date range: 2012-01-01 00:00:00 to 2019-08-01 00:00:00

Training set shape: (1491, 9)
Training date range: 2013-07-01 00:00:00 to 2017-07-31 00:00:00
Training period: 01.07.2013 to 31.07.2017

Validation set shape: (363, 9)
Validation date range: 2017-08-01 00:00:00 to 2018-07-31 00:00:00
Validation period: 01.08.2017 to 31.07.2018

Test set shape: (1840, 9)
Test date range: 2018-08-01 00:00:00 to 2019-07-31 00:00:00
Test period: 01.08.2018 to 31.07.2019

Expected total days: 2222
Actual total days in splits: 3694

Datasets saved:
- Training set: 'train_set.csv'
- Validation set: 'validation_set.csv'
- Test set: 'test_set.csv'

TRAINING SET - First 5 rows:
     id      Datum  Warengruppe  Umsatz  Bewoelkung  Temperatur  \
394 NaN 2013-07-01          NaN     NaN         6.0     17.8375   
395 NaN 2013-07-02          NaN     NaN         3.0     17.3125   
396 NaN 2013-07-03          NaN     NaN         7.0     21.0750   
397 NaN 20